In [57]:
from groq import Groq
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests
import tiktoken
from decouple import AutoConfig, config

In [58]:
config = AutoConfig(search_path='./')

In [59]:
client = Groq(api_key=config('API-KEY'))

In [60]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [61]:
documents[-1]

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

In [62]:
def build_prompt(query, search_results, section):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        if section is True:
            context = context + f"S: {doc['section']}\nQ: {doc['question']}\nA: {doc['text']}\n\n"
        else:
            context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"    
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [63]:
def llm(prompt):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
    )

    return chat_completion.choices[0].message.content

In [64]:
es_client = Elasticsearch('http://localhost:9200') 

In [65]:
!curl localhost:9200

{
  "name" : "fe8187192728",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "dFPPnYgQQoSKEvHE-sLhQA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [66]:
def elastic_search(query, course, size, rank):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text",],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    score_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        score_docs.append(hit['_score'])
    
    print(f"source: {result_docs[rank-1]}")
    print(f"rank: {score_docs[rank-1]}")

    return result_docs

In [71]:
def rag(query, course = "data-engineering-zoomcamp", size = 5, rank = 1, section = False):
    search_results = elastic_search(query, course, size, rank)
    prompt = build_prompt(query, search_results, section)
    print(f'prompt:\n {prompt[:400]}')
    print(f'prompt length: {len(prompt)}')
    answer = llm(prompt)
    return answer

In [72]:
query = 'How do execute a command on a Kubernetes pod?'

rag(query, section = True)

source: {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:', 'section': 'Course Management Form for Homeworks', 'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key', 'course': 'data-engineering-zoomcamp'}
rank: 31.973522
prompt:
 You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTI

'There is no information in the provided context on how to execute a command on a Kubernetes pod. The context only discusses topics related to Data Engineering, MLOps, Machine Learning, PySpark, and analytics engineering with dbt, but does not mention Kubernetes.'

In [73]:
query = "How do copy a file to a Docker container?"

rag(query, "machine-learning-zoomcamp", 3, 3)

source: {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}
rank: 59.812744
prompt:
 You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT:
    Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
doc
prompt length: 1462

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows: \n`docker cp /path/to/local/file_or_directory container_id:/path/in/container` \n\nReplace `/path/to/local/file_or_directory` with the path to the file you want to copy, `container_id` with the ID of the Docker container, and `/path/in/container` with the path where you want to copy the file in the container.'

In [74]:
encoding = tiktoken.encoding_for_model("gpt-4o")

query = "How do copy a file to a Docker container?"

search_results = elastic_search(query, "machine-learning-zoomcamp", 3, 3)

prompt = build_prompt(query, search_results, False)

tokens = encoding.encode(prompt)

print(tokens[:30])

len(tokens)

source: {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}
rank: 59.812744
[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 271, 7649, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959]


324

In [75]:
encoding.decode(tokens)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do copy a file to a Docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [76]:
answer = llm(prompt)

answer

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows: \ndocker cp /path/to/local/file_or_directory container_id:/path/in/container \n\nYou need to replace `/path/to/local/file_or_directory` with the path to the file you want to copy, `container_id` with the ID of the running Docker container, and `/path/in/container` with the path where you want to copy the file in the container.'

Llama-3.3-70B-Versatile is Meta's advanced multilingual large language model, optimized for a wide range of natural language processing tasks. With 70 billion parameters, it offers high performance across various benchmarks while maintaining efficiency suitable for diverse applications.

Input Token Price	$0.59 per 1M tokens

Output Token Price	$0.79 per 1M tokens

In [84]:
# Define pricing
INPUT_COST_PER_TOKEN = 0.59 / 1000000  # $ per token
OUTPUT_COST_PER_TOKEN = 0.79 / 1000000


def count_tokens(text: str) -> int:
    return len(encoding.encode(text))

def calculate_cost(input_text: str, output_text: str) -> dict:
    input_tokens = count_tokens(input_text)
    output_tokens = count_tokens(output_text)

    input_cost = input_tokens * INPUT_COST_PER_TOKEN
    output_cost = output_tokens * OUTPUT_COST_PER_TOKEN
    total_cost = input_cost + output_cost

    return {
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "input_cost": f"${round(input_cost, 6)}",
        "output_cost": f"${round(output_cost, 15)}",
        "total_cost": f"${round(total_cost, 6)}"
    }

In [85]:
input_text = prompt

output_text = answer

cost_info = calculate_cost(input_text, output_text)

cost_info

{'input_tokens': 324,
 'output_tokens': 99,
 'input_cost': '$0.000191',
 'output_cost': '$7.821e-05',
 'total_cost': '$0.000269'}